## USGS API
Source: https://data.usgs.gov/datacatalog/data/USGS:64d15b1dd34ef477cf3c0228

In [1]:
import requests
import os

# Define the base URL and item ID
BASE_URL = "https://www.sciencebase.gov/catalog/item/"
item_id = "64d15b1dd34ef477cf3c0228"
item_url = f"{BASE_URL}{item_id}?format=json"
save_dir = f"data/{item_id}"
os.makedirs(save_dir, exist_ok=True)

# Fetch metadata
response = requests.get(item_url)
response.raise_for_status()  # Raise an error for failed requests
data = response.json()

# Extract the file download URL
desired_file_path = "/catalog/file/get/64d15b1dd34ef477cf3c0228?f=__disk__7f%2F82%2Fa2%2F7f82a2ebace8c7dad328776faea0720d94f80968"
download_url = f"https://www.sciencebase.gov{desired_file_path}"

# Download the file
file_response = requests.get(download_url, stream=True)
file_response.raise_for_status()

# Save to local file
file_name = "LAS_Files.zip"
file_path = os.path.join(save_dir, file_name)
with open(file_path, "wb") as file:
    for chunk in file_response.iter_content(chunk_size=8192):
        file.write(chunk)

print(f"File downloaded successfully: {file_name} from {download_url}")

File downloaded successfully: LAS_Files.zip from https://www.sciencebase.gov/catalog/file/get/64d15b1dd34ef477cf3c0228?f=__disk__7f%2F82%2Fa2%2F7f82a2ebace8c7dad328776faea0720d94f80968


In [2]:
# Import the necessary libraries
import lasio, zipfile, os
import pandas as pd
import matplotlib.pyplot as plt

# Define the path to the downloaded LAS files
las_path = "data/64d15b1dd34ef477cf3c0228/LAS_Files"
zip_file = "data/64d15b1dd34ef477cf3c0228/LAS_Files.zip"

# Unzip the downloaded files
with zipfile.ZipFile(zip_file, "r") as zip_ref:
    zip_ref.extractall("/".join(zip_file.split("/")[:-1]))

# Scan for LAS files
las_files = []
for root, dirs, files in os.walk(las_path):
    for file_name in files:
        if file_name.endswith(".LAS"):
            las_files.append(os.path.join(root, file_name))

# Print the number of LAS files found
print(f"Found {len(las_files)} LAS files")

# Merge all las files into one dataframe
las_df = lasio.read(las_files[0]).df().reset_index()
for path in las_files[1:]:
    _df = lasio.read(path).df().reset_index()
    _df.drop(columns=set(_df.columns) & set(las_df.columns) - {'DEPT'}, inplace=True)
    
    las_df = las_df.merge(_df, on='DEPT', how='outer')

chosen_curves = ['DEPT', 'DEN(SS)', 'DEN(LS)', 'SANG', 'PAD1-AZ', 
                 'AMP 1', 'TEMP', 'MAG-VECT', 'GAMMA', 
                 'TENSION','TOOLFACE', 'GYRO', 'CAGE', 'NEUTRON', 'POR(NEU)']
las_df = las_df.loc[:, las_df.columns.intersection(chosen_curves)]

print(f"Merged {len(las_files)} LAS files to final dataframe")

las_df.head()

Found 7 LAS files
Merged 7 LAS files to final dataframe


,DEPT,DEN(SS),DEN(LS),SANG,PAD1-AZ,AMP 1,TEMP,MAG-VECT,GAMMA,TENSION,TOOLFACE,GYRO,CAGE,NEUTRON,POR(NEU)
0,-4.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,195.7,31.8,-999.0,NaN,NaN
